In [ ]:
from nltk.corpus import treebank
from nltk import PCFG
from nltk import treetransforms
from nltk import induce_pcfg
from nltk.parse import pchart

train_set = treebank.fileids()[:190]
test_set = treebank.fileids()[190:]

## Question 2
Créer une grammaire avec les données de trainng

In [ ]:
productions = []
probabilities = []
for item in train_set:
    for tree in treebank.parsed_sents(item):
        # perform optional tree transformations, e.g.:
        tree.collapse_unary(collapsePOS = False) # Remove branches A-B-C into A-B+C
        tree.chomsky_normal_form(horzMarkov = 2) # Remove A->(B,C,D) into A->B,C+D->D
        productions += tree.productions()
        # probabilities += tree.productions().prob()
print(productions[0])

In [ ]:
import heapq
from collections import Counter

pct= 10
nb_rules_init = len(productions)
nb_rules=int(nb_rules_init* pct/100)

nb_rules = 5000
productions_largest = list(dict(heapq.nlargest(nb_rules, dict(Counter(productions)).items(), key=lambda x: x[1])))

S = Nonterminal('S')
grammar = induce_pcfg(S,productions_largest)

## Question 3
Mesurer la grammaire créer 

In [ ]:
from nltk.parse import viterbi
from nltk.parse import ViterbiParser
import pickle
import time

Get each left hand side element of the grammar that is associated to a terminal word

In [ ]:
production_grammar = grammar.productions()
non_terminals = []
terminal = []

for rule in production_grammar:
    # if the right hand side is string, that means we need the left hand side
    # to add a rule for unknown words
    if(isinstance(rule.rhs()[0], str)):
        non_terminals.append(rule.lhs())

Add word for each 

In [ ]:
import re
new_production = productions_largest
missing_list = []
for item in test_set:
    for tree in treebank.parsed_sents(item):
        # go through each words
        for word in tree.leaves():
            # is the word in the grammar
            try:
                grammar.check_coverage([word])
            # if word is not in the grammar
            except Exception as missing:
                # print(word)
                for lhs in non_terminals:
                    # append to the grammar
                    rhs_new=['UNK']
                    new_production.append(nltk.grammar.Production(lhs,rhs_new))
                missing_list.append(word)

                
missing_set = set(missing_list)
print(len(missing_set))
S = Nonterminal('S')
new_grammar = induce_pcfg(S,new_production)
# print(new_grammar)

Create test set with UKN 

In [ ]:
known_words = []
for rule in new_grammar.productions():
    # if the right hand side is string, that means we need the left hand side
    # to add a rule for unknown words
    if(isinstance(rule.rhs()[0], str)):
        known_words.append(rule.rhs()[0])

In [ ]:
test_set_clean = []

#C,est bon maintenant

for item in test_set:
    for tree in treebank.parsed_sents(item):
        sentence = []
        # print(tree)
        for word in tree.leaves():
            if word not in known_words:
                sentence.append("UNK")
            else:
                sentence.append(word)
        test_set_clean.append(sentence)


In [ ]:
viberbi_parser = ViterbiParser(new_grammar)
probability_tree = viberbi_parser.parse_all(test_set_clean[0])

#pickle.dump(probability_tree, open("prob_tree.pkl", "wb" ) )
# pickle.load( open( "prob_tree.pkl", "rb" ) )

# Question 4

In [ ]:
from nltk import Tree

viberbi_parser = ViterbiParser(new_grammar)

tagger_preds_list = []
tagger_test_list = []
length_sentence = []
time_length_sentence = []

# for item in test_set:
for item in test_set[:1]:
    for tree in treebank.parsed_sents(item):

        start =  time.time()

        # add the UNK words in the sentence
        sentence = []
        for word in tree.leaves():
            if word not in known_words:
                sentence.append("UNK")
            else:
                sentence.append(word)


        test_set_clean.append(sentence)

        # Viterbi parser
        viberbi_probability_tree = viberbi_parser.parse_all(sentence)
        try:
            tagger_preds = viberbi_probability_tree[0].pos()
            tagger_preds_list.append(tagger_preds)

        except:
            tagger_preds_list.append([])

        tagger_test= tree.pos()

        end = time.time()

        # add data to arrays
        length_sentence.append(len(sentence))
        time_length_sentence.append(end-start)
        
        tagger_test_list.append(tagger_test)

        # save the arrays
        pickle.dump(length_sentence, open("length_sentence.pkl", "wb" ) )
        pickle.dump(time_length_sentence, open("time_length_sentence.pkl", "wb" ) )
        pickle.dump(tagger_preds_list, open("tagger_preds_list.pkl", "wb" ) )
        pickle.dump(tagger_test_list, open("tagger_test_list.pkl", "wb" ) )

        
tagger_preds_list = pickle.load( open( "tagger_preds_list.pkl", "rb" ) )
tagger_test_list = pickle.load( open( "tagger_test_list.pkl", "rb" ) )      

# print(tagger_preds_list)
# print(tagger_test_list)



In [ ]:
from sklearn import metrics

tests = [tag for sentence in tagger_test_list for _,tag in sentence]
preds = [tag for sentence in tagger_preds_list for _,tag in sentence]

print("Metrics")
print(" Accuracy :", metrics.accuracy_score(tests,preds))
print(" Precision:", metrics.precision_score(tests,preds,average='weighted'))
print(" Recall   :", metrics.recall_score(tests,preds,average='weighted'))
print(" F1-Score :", metrics.f1_score(tests,preds,average='weighted'))
print()

pickle.load( open( "time_length_sentence.pkl", "rb" ) )  

# Question 5

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics

In [ ]:
list_length_sentence = pickle.load(open( "length_sentence_5k.pkl", "rb" ))
list_time = pickle.load(open( "time_length_sentence_5k.pkl", "rb" ))
tagger_preds = pickle.load(open( "tagger_preds_list_5k.pkl", "rb" ))
tagger_gold = pickle.load(open( "tagger_test_list_5k.pkl", "rb" ))

index = [i for i in range(0,226)]
even_nos = [num for num in index if num % 2 == 0] 
tagger_preds = np.array(tagger_preds)[even_nos]
tagger_preds = list(tagger_preds)

new_tagger_preds = []
new_tagger_gold = []

for i,sentence in enumerate(tagger_preds):

    if(len(tagger_gold[i]) == len(tagger_preds[i])):
        new_tagger_gold.append(tagger_gold[i])
        new_tagger_preds.append(tagger_preds[i])

tagger_preds = new_tagger_preds
tagger_gold = new_tagger_gold

list_length_sentence_25 = pickle.load(open( "result2/length_sentence.pkl", "rb" ))
list_time_25 = pickle.load(open( "result2/time_length_sentence.pkl", "rb" ))
tagger_preds_25 = pickle.load(open( "result2/tagger_preds_list.pkl", "rb" ))
tagger_gold_25 = pickle.load(open( "result2/tagger_test_list.pkl", "rb" ))

list_length_sentence_50 = pickle.load(open( "result3/length_sentence.pkl", "rb" ))
list_time_50 = pickle.load(open( "result3/time_length_sentence.pkl", "rb" ))
tagger_preds_50 = pickle.load(open( "result3/tagger_preds_list.pkl", "rb" ))
tagger_gold_50 = pickle.load(open( "result3/tagger_test_list.pkl", "rb" ))

list_length_sentence_100 = pickle.load(open( "result1/length_sentence.pkl", "rb" ))
list_time_100 = pickle.load(open( "result1/time_length_sentence.pkl", "rb" ))
tagger_preds_100 = pickle.load(open( "result1/tagger_preds_list.pkl", "rb" ))
tagger_gold_100 = pickle.load(open( "result1/tagger_test_list.pkl", "rb" ))

In [ ]:
order = np.argsort(list_length_sentence)
list_length_sentence = np.array(list_length_sentence)[order]
list_time = np.array(list_time)[order]
tagger_preds = np.array(tagger_preds)[order]
tagger_gold = np.array(tagger_gold)[order]

order = np.argsort(list_length_sentence_50)
list_length_sentence_50 = np.array(list_length_sentence_50)[order]
list_time_50 = np.array(list_time_50)[order]
tagger_preds_50 = np.array(tagger_preds_50)[order]
tagger_gold_50 = np.array(tagger_gold_50)[order]

order = np.argsort(list_length_sentence_100)
list_length_sentence_100 = np.array(list_length_sentence_100)[order]
list_time_100 = np.array(list_time_100)[order]
tagger_preds_100 = np.array(tagger_preds_100)[order]
tagger_gold_100 = np.array(tagger_gold_100)[order]

Longuuer phrase et temps

In [ ]:
plt.plot(list_length_sentence, list_time/60, "-", color = '#244864', markersize=3, label = "2.5% rules")
#plt.plot(list_length_sentence_25, list_time_25/60, "-", color = '#34e621', markersize=3, label = "25% rules")
plt.plot(list_length_sentence_50, list_time_50/60, "-", color = '#871f78', markersize=3, label = "50% rules")
plt.plot(list_length_sentence_100, list_time_100/60, "-", color = '#ff0000', markersize=3, label = "100% rules")

plt.xlabel('Longueur de phrases [mots]', fontsize = 10)
plt.ylabel('Temps [min]', fontsize = 10)
plt.grid()
plt.legend()
plt.savefig('time_length.png')

In [ ]:
def average(lst): 
    return sum(lst) / len(lst) 

avg_time = [average(list_time), average(list_time_25), average(list_time_50), average(list_time_100)]
nb_rules_initial = [5000, 0.25*, average(list_time_50), average(list_time_100)]

plt.plot(list_length_sentence, list_time, "o", color = '#244864', markersize=5)

plt.xlabel('Longueur de phrases [mots]', fontsize = 10)
plt.ylabel('Temps [s]', fontsize = 10)
plt.grid()

Longueur phrase et justesse

In [1]:
# missing_list = pickle.load( open( "missing_list.pkl", "rb" ) )
def get_accuracy_list(tagger_preds, tagger_gold):

    accuracy_each_sentence = []

    for i,sentence in enumerate(tagger_preds):

        tests = [tag for _,tag in tagger_gold[i]]
        preds = [tag for _,tag in tagger_preds[i]]
        
        acc = metrics.accuracy_score(tests,preds)
        accuracy_each_sentence.append(acc)

    #print(" Accuracy :", metrics.accuracy_score(tests,preds))
    #print(" Precision:", metrics.precision_score(tests,preds,average='weighted'))
    #print(" Recall   :", metrics.recall_score(tests,preds,average='weighted'))
    #print(" F1-Score :", metrics.f1_score(tests,preds,average='weighted'))
    return accuracy_each_sentence

In [ ]:
plt.plot(list_length_sentence_50, acc_list_50, "o", color = '#871f78', markersize=3, label = "50% rules")

plt.xlabel('Longueur de phrases [mots]', fontsize = 10)
plt.ylabel('Justesse', fontsize = 10)
plt.grid()
#plt.legend()
plt.savefig('acc_length.png')

Proportion de mots inconnus par phrase

In [ ]:
# for item in test_set:
count_unk_list = []
for item in test_set:
    for tree in treebank.parsed_sents(item):
        count_unk = 0
        start =  time.time()

        # add the UNK words in the sentence
        sentence = []
        for word in tree.leaves():
            if word not in known_words:
                sentence.append("UNK")
                count_unk += 1
            else:
                sentence.append(word)
        count_unk_list.append(count_unk/len(sentence))

print(len(count_unk_list))


tagger_preds_100 = pickle.load(open( "result1/tagger_preds_list.pkl", "rb" ))
tagger_gold_100 = pickle.load(open( "result1/tagger_test_list.pkl", "rb" ))

order = np.argsort(count_unk_list[:67])
list_length_unk_100 = np.array(count_unk_list)[order]
tagger_preds_unk_100 = np.array(tagger_preds_100)[order]
tagger_gold_unk_100 = np.array(tagger_gold_100)[order]
acc_list_100 = get_accuracy_list(tagger_preds_unk_100, tagger_gold_unk_100)

plt.plot(list_length_unk_100, acc_list_100, "o", color = '#871f78', markersize=3, label = "100% rules")

plt.xlabel('Proportion de mots inconnus par phrase [mots]', fontsize = 10)
plt.ylabel('Justesse', fontsize = 10)
plt.grid()
plt.savefig('acc_unk_prop.png')